# DuckDB Handout: Working with Remote Data (HTTPS)

(**Note:** The examples were created by Gemini, based on provided instructions. All the text and code included here was checked and simplified by EM.)

Welcome back! You are familiar with SQL, and DuckDB uses the same language, but we are now shifting from local SQLite to working with a powerful, remote analytical database file.

### Google Colab
The use of Google Colab is optional. It's useful if it is hard to install **duckdb** on your computer.

### DuckDB Context: The remote database file

The file we are using is a `.duckdb` file, which is an entire, pre-built database (similar to an SQLite .db file). Unlike SQLite, DuckDB can connect to this file over an HTTPS server and allow us to run queries read-only without having to download the entire database first.

### Wikipedia DPDP

The database we are working with collects the 2024 page views for the English wikipedia in the United States.

## 1. Setup in Google Colab

In case the package is not installed, we do so below.

In [ ]:
!pip install duckdb

We make sure that it is installed:

In [ ]:
import duckdb

# Placeholder for the database connection. It will be initialized later with the URL.
conn = duckdb.connect()
conn

### Enable HTTPFS

Since our database is accessed via a URL (over the web), we need to load the built-in HTTPFS (HTTP File System) extension within our DuckDB session. This allows DuckDB to read files hosted on remote servers.

In [ ]:
# Install and Load the HTTPFS extension
# This is required to access remote files over the web (HTTP/S)
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

## 2. Connect to the remote database source

We do not connect to a remote DuckDB file in the traditional client-server sense. Instead, we ATTACH the remote file as a database to our current local session. DuckDB then streams the necessary data over HTTP.

In [ ]:
# This is one of the several DuckDB databases hosted in the CS server.
# This database had the DPDP data for the year 2024 for the United States (en).
database_url = "https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb"

# Attach the remote file as a database named 'web_db' and start using it
try:
    conn.execute(f"ATTACH '{database_url}' AS web_db (READ_ONLY);")
    conn.execute("USE web_db;")
    print(f"Successfully attached database from: {database_url}")
except Exception as e:
    print(f"Error attaching database: {e}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Successfully attached database from: https://cs.wellesley.edu/~eni/duckdb/2024_wiki_views.duckdb


## 3. Show the tables

Now that the remote database is attached, we can query its catalog to see what tables it contains. While we know the command "SELECT" for running queries on the table, the command "PRAGMA" (which means action in Greek) operates at the level of the database itself.

Below we will always adapt a two-step strategy:
1. formulate the string of a query
2. Run the query through the command: `conn.sql`

In [ ]:
query = "PRAGMA show_tables"
result = conn.sql(query)
result

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ data_table │
└────────────┘

Our database has only one table which is named "data_table". We will write another PRAGMA query to check on the column names of this table.

In [ ]:
table_name = "data_table"
query = f"PRAGMA table_info('web_db.{table_name}');"

# We can apply the method .df() to the result of the query to convert it into a dataframe
column_info_df = conn.sql(query).df()
column_info_df

,cid,name,type,notnull,dflt_value,pk
0,0,date,DATE,False,None,False
1,1,country,VARCHAR,False,None,False
2,2,country_code,VARCHAR,False,None,False
3,3,project,VARCHAR,False,None,False
4,4,page_id,BIGINT,False,None,False
5,5,article,VARCHAR,False,None,False
6,6,qid,VARCHAR,False,None,False
7,7,pageviews,BIGINT,False,None,False


## 4. SQL commands for the table

Now that we know that names of the columns of the table, we can formulate various queries to get data from the table.

### 4.1 Select a few rows

Use SELECT * to get every column, and LIMIT to restrict the number of rows returned, which is essential when working with large remote files.

In [ ]:
query_1 = """
SELECT * FROM data_table
LIMIT 10;
"""
result_1 = conn.sql(query_1).df() # after executing, convert to df for better printout

result_1

,date,country,country_code,project,page_id,article,qid,pageviews
0,2024-01-01,United States,US,en.wikipedia,100580,"Selma,_Alabama",Q79941,117
1,2024-01-01,United States,US,en.wikipedia,10058256,Jimmy_John_Liautaud,Q24041548,143
2,2024-01-01,United States,US,en.wikipedia,1008381,Cheerwine,Q5089372,300
3,2024-01-01,United States,US,en.wikipedia,10095918,New_York_Fashion_Week,Q1113829,117
4,2024-01-01,United States,US,en.wikipedia,10122474,Stephanie_Edwards,Q7608269,195
5,2024-01-01,United States,US,en.wikipedia,101615,Tocumen_International_Airport,Q1420178,117
6,2024-01-01,United States,US,en.wikipedia,1016778,Ãrebro,Q25732,123
7,2024-01-01,United States,US,en.wikipedia,1023183,Kidd-class_destroyer,Q246916,124
8,2024-01-01,United States,US,en.wikipedia,1027629,Long_Duk_Dong,Q55621207,305
9,2024-01-01,United States,US,en.wikipedia,10338,Excommunication,Q49822,237


### 4.2 Select specific columns

Specify only the columns we need, which is more efficient, as DuckDB only fetches data for those columns over the web.

In [ ]:
query_2 = """
SELECT article, country, pageviews
FROM data_table
LIMIT 5;
"""
result_2 = conn.sql(query_2).df()

result_2

,article,country,pageviews
0,"Selma,_Alabama",United States,117
1,Jimmy_John_Liautaud,United States,143
2,Cheerwine,United States,300
3,New_York_Fashion_Week,United States,117
4,Stephanie_Edwards,United States,195


### 4.3 Filter rows by string values

The WHERE clause lets us apply conditions to filter the rows based on column values.

In [ ]:
query_3 = """
SELECT date, pageviews
FROM data_table
WHERE article = 'Long_Duk_Dong'
LIMIT 5;
"""
result_3 = conn.sql(query_3).df()
result_3

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,date,pageviews
0,2024-01-01,305
1,2024-01-02,155
2,2024-01-03,481
3,2024-01-04,139
4,2024-01-05,105


We found five dates and the respective pageviews for the article "Long_Duk_Dong".

### 4.4 Filter rows by numeric values

Find articles with more than 10,000 views per day.

In [ ]:
query_4 = """
SELECT date, article, pageviews
FROM data_table
WHERE pageviews > 10000
LIMIT 5;
"""

result_4 = conn.sql(query_4).df()
result_4

,date,article,pageviews
0,2024-01-01,Oppenheimer_(film),27482
1,2024-01-01,John_Harbaugh,28751
2,2024-01-01,Saltburn_(film),125742
3,2024-01-01,Elizabeth_II,26727
4,2024-01-01,Mary_Kay_Letourneau,16862


### 4.5 Sorting columns using `ORDER BY`

Sort the table by the values of a providing column. Notice how we can change the order of the columns through the SELECT command.

In [ ]:
query_5 = """
SELECT article, pageviews, date
FROM data_table
ORDER BY pageviews DESC
LIMIT 15;
"""
result_5 = conn.sql(query_5).df()

result_5

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,article,pageviews,date
0,Main_Page,4120840,2024-11-06
1,Main_Page,3493333,2024-07-28
2,J.D._Vance,3397556,2024-07-15
3,Main_Page,3279912,2024-07-27
4,Anthropology,3253747,2024-12-25
5,Main_Page,2648299,2024-07-10
6,Gwen_Walz,2393644,2024-08-06
7,Main_Page,2212819,2024-04-08
8,Hegseth,2146555,2024-11-13
9,J.D._Vance,2071310,2024-07-16


One important takeway from this table is that the most viewed page on most days is the "Main_Page", because people visit Wikipedia's main landing page before searching for an article. This also tells us that we should not include this page when we perform data analysis.

### 4.6 Aggregation using `GROUP BY`

GROUP BY is used with aggregate functions (like SUM, COUNT, AVG, MAX) to summarize data across groups.

The following query is trying to find articles in the database that have been viewed on most days.

**Important:** Notice the syntax for the SELECT command. We can use it with column names that don't exist yet and are created on the fly.

In [ ]:
query_6 = """
SELECT
    article,
    COUNT(date) AS number_of_days
FROM data_table
GROUP BY article
ORDER BY number_of_days DESC
LIMIT 10;
"""
result_6 = conn.sql(query_6).df()
result_6

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,article,number_of_days
0,Autism,405
1,Lionsgate,368
2,Brian_Thompson,368
3,Super_Sentai,368
4,1,368
5,Big_Boss_(Metal_Gear),368
6,Edo,368
7,Mehdi_Hasan,368
8,2012_United_States_presidential_election,367
9,Dora_the_Explorer,367


We see some problematic values here. The year of 2024 had 366 days, so these values should not be bigger than 366, but there might have been an issue when the database was created.

Now let's find the top 10 articles based on their pageviews:

In [ ]:
query_7 = """
SELECT
    article,
    SUM(pageviews) AS total_pageviews
FROM data_table
GROUP BY article
ORDER BY total_pageviews DESC
LIMIT 10;
"""

result_7 = conn.sql(query_7).df()
result_7

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,article,total_pageviews
0,Main_Page,548196381.0
1,Kamala_Harris,15556259.0
2,Project_2025,13418101.0
3,J.D._Vance,10270384.0
4,Biggest_ball_of_twine,9838519.0
5,Erik_Menendez,8506713.0
6,Raindrop_cake,8386713.0
7,Kepler's_Supernova,8019858.0
8,Deadpool_&_Wolverine,7529702.0
9,Elon_Musk,7228156.0


## 5. Interactive Querying

We'll create a notebook widget to allow us select months from the year.



In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create a list of month numbers (1 to 12) for the dropdown options
month_options = [(str(m), m) for m in range(1, 13)]

# Create the Dropdown widget
month_selector = widgets.Dropdown(
    options=month_options,
    value=1,  # Default to January
    description='Select Month:',
    disabled=False,
)

We'll write a function that the widget will call to show top 10 articles for one month.

In [ ]:
def get_top_articles_for_month(month_num):
    """
    Runs the DuckDB query for the selected month and prints the results.
    """

    # Define the article to exclude
    EXCLUDE_ARTICLE = 'Main_Page'

    # Use the MONTH() SQL function to filter by month number (1-12)
    query = f"""
    SELECT
        article,
        SUM(pageviews) AS total_pageviews
    FROM data_table
    WHERE MONTH(date) = {month_num}
      AND article <> '{EXCLUDE_ARTICLE}'
    GROUP BY article
    ORDER BY total_pageviews DESC
    LIMIT 10;
    """

    result_df = conn.sql(query).df()
    display(result_df)

Finally, this is the code that will run the interactive widget. Whenever you select a different month number, it will run the query and update the table of the top 10 tables. Since the database is on the server, there is a bit of delay, but not much.

In [ ]:
interactive_output = widgets.interactive(get_top_articles_for_month,
                                         month_num=month_selector)

# Display the widget and the output area in the notebook
display(interactive_output)


interactive(children=(Dropdown(description='Select Month:', index=11, options=(('1', 1), ('2', 2), ('3', 3), (…